# Preparations

Execute the following code blocks to configure the session and import relevant modules.

In [ ]:
%config InlineBackend.figure_format ='retina'
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import sys
import math
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

There is also a utility module [rnnutils.py](https://raw.githubusercontent.com/NBISweden/workshop-neural-nets-and-deep-learning/master/session_recurrentNeuralNetworks/lecture_RNN/rnnutils.py) that you can use if you want to save time coding. Whenever it is used, it will be commented out to leave you the choice whether you want to use the functions or write up your own solution. In any case, make sure the file is located in the current directory to load it.

In [ ]:
import rnnutils

# Lab session: predicting airline passengers

## Aims

In this lab the idea is to try out different RNN models on the Box & Jenkins monthly airline passengers dataset. The dataset is a monthly time series of airline passengers recorded in the 50'ies and 60'ies. Your task is to build a model to make a future prediction of the number of passengers given a number of observation.

You will download data and prepare it for later analyses. More specifically, you will partition the data into a training and test set. In order to create input / label pairs (X/Y), the data is split into time slices, where a slice corresponds to the input (X) and the consecutive time point the (known) output (Y).


To help you along the way, some of the steps have been prepared in advance, but in most cases, your task is to complete missing code. Don't hesitate to change parameter settings and experiment with the model architectures. Also, make sure to examine the contents of variables by printing them. Things to try:

- change the number of time steps
- change the number of epoch
- experiment with the network topology (e.g. number of units in the hidden layer)

See if you can improve on the model presented in the lecture.

# Session 1: Vanilla RNN

## Download data

Start by downloading the data and loading it into a pandas dataframe:

In [ ]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv --no-check-certificate

We modify the data somewhat for easier processing. `df.head()` simply shows you the first entries of the data frame.

In [ ]:
df = pd.read_csv('airline-passengers.csv')
df = df.rename(columns={'Month': 'time','Passengers': 'passengers'})
df['time'] = pd.to_datetime(df['time'], format='%Y-%m')
df['year'] = pd.DatetimeIndex(df['time']).year
df['month'] = pd.DatetimeIndex(df['time']).month
df.head()

Plot the data for overview:

In [ ]:
plt.plot(df.time, df.passengers)

## Create training and test data


Next, we partition the data into training and test data sets. We first define the size of the training set and the index at which to split the data.

In [ ]:
train_fraction = 2/3
split_index = int(df.shape[0] * train_fraction)

The [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) function rescales (normalizes) the data to values in the range (0, 1) with unit variance, the motivation being that the network training tends to perform better in this case. Here, it is important to remember that scaling should be done first on the training data, and then that the same transformation is applied to the test data. In practice, we first rescale the training data independently. Then, the resulting scaler instance is used to rescale both the test data and the entire data set.

In [ ]:
# Reshape data for MinMaxScaler
data = np.array(df['passengers'].values.astype('float32')).reshape(-1, 1)
# Instantiate scaler object
scaler = MinMaxScaler(feature_range=(0, 1))
# Scale training data
train = data[range(split_index)]
train_scaled = scaler.fit_transform(train).flatten()
# Apply *same* transformation to test data
test = data[split_index:]
test_scaled = scaler.transform(test).flatten()

## Transform data to input - output pairs


Now that we have train and test data sets we need to convert the data to input - output (X/Y) pairs. The general idea is to take time slices (e.g. 12 data points) as input vectors and use the subsequent value as the known output. Since the time unit is months and there likely is a recurrent yearly seasonality in the data it makes sense to use 12 time steps, but this is a parameter you could modify to see what effect it has on the end results. In particular, if you increase this number, you would look further back in the past when making predictions.

In [ ]:
time_steps = 12
trainX, trainY, trainX_indices, trainY_indices = rnnutils.make_xy(train_scaled, time_steps)
testX, testY, testX_indices, testY_indices = rnnutils.make_xy(test_scaled, time_steps) 

## Define the model


Complete the model below to include a [SimpleRNN](https://keras.io/api/layers/recurrent_layers/simple_rnn/) layer and a [Dense](https://keras.io/api/layers/core_layers/dense/) output layer. If you look at the SimpleRNN documentation, you will find that inputs is a 3D tensor (`[batch, timesteps, feature]`). Since we are using univariate data (i.e. one feature per time step), `features=1`. Recall also that for the `input_shape` parameter you don't specify `batch`. You can click the '+' button to the top left to reveal a working example, but first try to complete the model without peeking! Also, consult the lecture notes for an example.

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=4, input_shape=(time_steps, 1), activation="tanh"))
model.add(Dense(units=1))
model.compile(loss="mse", optimizer="adam")
model.summary()

In [ ]:
# model = Sequential()
# Add layers here
# model.add()
#
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# model.summary()

Once you are happy with the configuration, fit the model and evaluate. 

In [ ]:
history = model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2, validation_split=0.1)
Ytrainpred = model.predict(trainX)
Ytestpred = model.predict(testX)

In [ ]:
# history = model.fit(trainX, trainY, ...)
# Ytrainpred = model.predict(trainX)
# Ytestpred = model.predict(testX)

You can use the utility plotting functions in `rnnutils` to plot training history and predictions

In [ ]:
rnnutils.plot_history(history, figsize=(14, 8))

In [ ]:
data = {'train': (Ytrainpred, train_scaled, trainY_indices),
       'test': (Ytestpred, test_scaled, testY_indices)}
rnnutils.plot_pred(data, scaler=scaler, figsize=(14, 8), plotmarkers=True)







# Session 2: LSTM (and optionally GRU) 

Building on session 1, analyse the data set using LSTM layers. Here is a tentative model setup to get you started. Here you could try using multiple layers, in which case you need to return the sequences for all but the last layer (cf [Stacked Long Short-Term Memory Networks](https://machinelearningmastery.com/stacked-long-short-term-memory-networks/)). If you have time, you can also try out the GRU layers for comparison. Do you notice any difference?

In [ ]:
model = Sequential()
model.add(LSTM(units=4, return_sequences=False, input_shape=(time_steps, 1)))
# Remember: if you stack LSTMs, remember to set return_sequences=True for all but the last layer
# model.add(LSTM(units=2, return_sequences=False))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
history = model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2, validation_split=0.1)
Ytrainpred = model.predict(trainX)
Ytestpred = model.predict(testX)

In [ ]:
rnnutils.plot_history(history, figsize=(14, 8))

In [ ]:
data = {'train': (Ytrainpred, train_scaled, trainY_indices),
       'test': (Ytestpred, test_scaled, testY_indices)}
rnnutils.plot_pred(data, scaler=scaler, figsize=(14, 8), plotmarkers=True)

Recall that the default setting for time steps is 12. If you increase this number, you would look further back in time, which in principle should play to the strength of LSTMs. Note that this need not necessarily lead to better predictions though.

In [ ]:
# model = Sequential()
# model.add(LSTM(..., return_sequences=True, input_shape=(..., ...)))
# model.add(LSTM(..., return_sequences=False))
# model.compile(loss='mean_squared_error', optimizer='adam')